In [2]:
import pandas as pd
from pprint import pprint
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine

class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):        
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                child_str = ' '.join(child_str.split()).strip()
                if child_str:
                    row = (page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str) # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage

fp = open("Goethe-Zertifikat_B1_Wortliste.pdf", 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)
#doc.initialize() # leave empty for no password

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

for page in PDFPage.create_pages(doc):
    interpreter.process_page(page)
    # receive the LTPage object for this page
    device.get_result()

In [3]:
def columntranslation(x):
    if x<129:
        return 1
    if x<300:
        return 2
    if x<410.5:
        return 3
    return 4

def assignblock(x,limits):
    return sum(x>=limits)

def addspace(x):
    output = x.iloc[0]
    for i in range(1,len(x)):
        xi = x.iloc[i]
        if len(output)>=2:
            if (output[-1]=="-") & (output[-2].isalpha()):
                output=output[:-1]+xi
        output+=" "+xi
            
    return output

In [4]:
DF = pd.DataFrame(device.rows)
DF.columns = ["page","xmin","ymin","xmax","ymax","text"]
alphw = DF[(DF["page"]>14)&(DF["page"]<102)&(DF["ymin"]>48.44)&(DF["ymin"]<807)]
alphw["column"]=alphw["xmin"].apply(columntranslation)
alphw["columnaux"]=1-alphw["column"].isin([1,3])
alphw["yabs"]=(850-alphw["ymax"]+850*(alphw["column"]>2.5)+alphw["page"].apply(int)*2000-30000).apply(lambda x: int(round(x,0)))

C:\Users\GiuliaAvvisati\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\GiuliaAvvisati\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\GiuliaAvvisati\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [5]:
entries = alphw[alphw["columnaux"]==0]
entries = entries.sort_values(["page","yabs"])
entries["dy"] = entries["yabs"].diff()
entries["groupaux"] = (entries["dy"]>13.05).cumsum()
limits = entries.groupby("groupaux").yabs.min()-4

alphw["block"] = alphw["yabs"].apply(lambda x: assignblock(x,limits))
alphw = alphw.sort_values(["columnaux","yabs","xmin"])#.reset_index(drop=True)
final = alphw.groupby(["columnaux","block"]).agg({"text":addspace}).reset_index(drop=False)

entries  = final[final["columnaux"]==0]
examples = final[final["columnaux"]==1]
ff = pd.merge(entries, examples, on="block")

C:\Users\GiuliaAvvisati\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
ff = ff[["text_x","text_y"]]
ff.columns = ["entries","examples"]

In [7]:
aux = list(ff["examples"])

In [9]:
translation_list = []

In [11]:
import time
print("prova")
time.sleep(2.5)
print("prova")

prova
prova


In [15]:
from google_trans_new import google_translator  
  
translator = google_translator()

In [16]:
for i in range(len(aux)):
    time.sleep(3)
    if i%10==0:
        time.sleep(5)
    if i%100==0:
        time.sleep(60)
        print(i)
        pd.Series(translation_list).to_pickle("B1translist.pkl")
    if i%500==0:
        time.sleep(120)
    translation_list.append(translator.translate(aux[i], lang_src='de', lang_tgt='en'))
    

0


C:\Users\GiuliaAvvisati\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [18]:
translation_list[:100]

['1. The ride costs 200 euros from Hamburg. 2. From next week our business will remain closed on Saturdays. 3. My brother visits us from time to time. ',
 'At the next intersection you have to turn left. ',
 'On the picture you can see how to turn on the device. ',
 '1. Our holiday was a great adventure. 2. The children love adventure stories. Expensive stories. ',
 '1. Today I can not, but tomorrow. 2. It was a lot of snow, but Enzo has nevertheless driven the motorcycle. 3. We only have a small apartment, but are satisfied with it. 4. It was very nice. But now I have to go. 5. I would like to come, but unfortunately it is not possible. 6. May I invite you to a coffee? - But yes, very much. 7. But you play a lot of piano. ',
 'Our train has dropped on time. ',
 'It stays twenty minutes until the departure. ',
 'Please do not throw the biowaste to normal waste. ',
 'Please throw that in the trash can! ',
 'Exhaust gases from industry and households pollute the air. ',
 'I should give t

In [17]:
ff["examples_trans"] = ff["examples"].apply(lambda x: translator.translate(x, lang_src='de', lang_tgt='en'))

google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [16]:
ff["examples"].iloc[:4].apply(lambda x: translator.translate(x, lang_src='de', lang_tgt='en'))

0    1. The ride costs 200 euros from Hamburg. 2. F...
1     At the next intersection you have to turn left. 
2    On the picture you can see how to turn on the ...
3    1. Our holiday was a great adventure. 2. The c...
Name: examples, dtype: object

In [ ]:
!pip install googletrans

In [ ]:
import translate

In [14]:
!pip install google_trans_new

In [13]:
import googletrans
translator = googletrans.Translator()
sentence = translator.translate("Hallo! Mein Name ist Merkel. Wie heisst du?",dest="en",src="ge")
sentence.extra_data

ValueError: invalid source language

In [11]:
translate.translator('de', 'en',"Hallo! Mein Name ist Merkel. Wie heisst du?")

NameError: name 'translate' is not defined

In [ ]:
ff

In [18]:
!pip install deep_translator

In [23]:
from deep_translator import PonsTranslator
word = 'my boss is hebrew but he is not kind'
translated_word = PonsTranslator(source='english', target='french').translate(word, return_all=False)

In [24]:
translated_word

'être '